<div align="center" id="top">
 <img src="https://socialify.git.ci/julep-ai/julep/image?description=1&descriptionEditable=Rapidly%20build%20AI%20workflows%20and%20agents&font=Source%20Code%20Pro&logo=https%3A%2F%2Fraw.githubusercontent.com%2Fjulep-ai%2Fjulep%2Fdev%2F.github%2Fjulep-logo.svg&owner=1&forks=1&pattern=Solid&stargazers=1&theme=Auto" alt="julep" width="640" height="320" />
</div>

<p align="center">
  <br />
  <a href="https://docs.julep.ai" rel="dofollow">Explore Docs (wip)</a>
  ·
  <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>
  ·
  <a href="https://x.com/julep_ai" rel="dofollow">𝕏</a>
  ·
  <a href="https://www.linkedin.com/company/julep-ai" rel="dofollow">LinkedIn</a>
</p>

<p align="center">
    <a href="https://www.npmjs.com/package/@julep/sdk"><img src="https://img.shields.io/npm/v/%40julep%2Fsdk?style=social&amp;logo=npm&amp;link=https%3A%2F%2Fwww.npmjs.com%2Fpackage%2F%40julep%2Fsdk" alt="NPM Version"></a>
    <span>&nbsp;</span>
    <a href="https://pypi.org/project/julep"><img src="https://img.shields.io/pypi/v/julep?style=social&amp;logo=python&amp;label=PyPI&amp;link=https%3A%2F%2Fpypi.org%2Fproject%2Fjulep" alt="PyPI - Version"></a>
    <span>&nbsp;</span>
    <a href="https://hub.docker.com/u/julepai"><img src="https://img.shields.io/docker/v/julepai/agents-api?sort=semver&amp;style=social&amp;logo=docker&amp;link=https%3A%2F%2Fhub.docker.com%2Fu%2Fjulepai" alt="Docker Image Version"></a>
    <span>&nbsp;</span>
    <a href="https://choosealicense.com/licenses/apache/"><img src="https://img.shields.io/github/license/julep-ai/julep" alt="GitHub License"></a>
</p>

## Task Definition: Email Assistant

### Overview

This task creates an email assistant that can receive emails, search through Julep documentation, and respond to user inquiries automatically. It combines email integration with documentation search capabilities to provide relevant and informed responses.

### Task Tools:

**send_email**: An `integration` type tool that handles email sending via Mailgun SMTP.
**search_docs**: A `system` type tool that searches through agent documentation.

### Task Input:

A dictionary containing:
- **from**: Sender's email address
- **to**: Recipient's email address
- **subject**: Email subject
- **body**: Email content

### Task Output:

An email response sent to the inquirer with:
- Generated subject line
- Generated response body based on documentation search results

### Task Flow

1. **Input**: Receive email details (from, to, subject, body)
2. **Query Generation**: Generate a search query based on the email content
3. **Documentation Search**: Search Julep documentation using the generated query
4. **Response Generation**: Create a response using the found documentation
5. **Email Sending**: Send the response back to the original sender via Mailgun

```plaintext
+----------+     +-------------+     +------------+     +-----------+
|  Email   |     |    Query    |     |    Doc     |     |  Email    |
|  Input   | --> | Generation  | --> |   Search   | --> | Response  |
| (Query)  |     |             |     |            |     | Output    |
+----------+     +-------------+     +------------+     +-----------+
      |                |                  |                  |
      |                |                  |                  |
      v                v                  v                  v
   "How do I"     Create search     Find relevant    "Here's how to
  "use Julep?"      keywords        documentation    get started..."
```

## Implementation

To recreate the notebook and see the code implementation for this task, you can access the Google Colab notebook using the link below:

<a target="_blank" href="https://colab.research.google.com/github/julep-ai/julep/blob/dev/cookbooks/00-Devfest-Email-Assistant.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Additional Information

For more details about the task or if you have any questions, please don't hesitate to contact the author:

**Author:** Julep AI  
**Contact:** [hey@julep.ai](mailto:hey@julep.ai) or  <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>

In [22]:
!pip install julep

## Creating Julep Client with the API Key

Get you API key from [here](https://dashboard.julep.ai/)

In [23]:
import os

import yaml
from julep import Julep
from dotenv import load_dotenv

load_dotenv(override=True)

api_key = os.getenv("JULEP_API_KEY")

julep = Julep(api_key=api_key, environment="production")

## Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the [documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#agent).

In [24]:
agent = julep.agents.create(
    name="Julep Email Assistant",
    about=(
        "You are an agent that handles emails for julep users."
        + " Julep is a platform for creating kick-ass AI agents."
    ),
    model="gpt-4o",
    default_settings={"temperature": 0.2},
)

In [25]:
agent.id

'067a5e5b-0651-7f64-8000-571e39fa9b30'

## Defining a Task

Tasks in Julep are Github-Actions-style workflows that define long-running, multi-step actions.

You can use them to conduct complex actions by defining them step-by-step.

To learn more about tasks, please refer to the `Tasks` section in [Julep Concepts](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#tasks).

In [ ]:
mailgun_password = os.getenv("MAILGUN_PASSWORD")

task_def = yaml.safe_load(f"""
name: Julep Email Assistant

input_schema:
  type: object
  properties:
    from:
      type: string
    to:
      type: string
    subject:
      type: string
    body:
      type: string

tools:
- name: send_email
  type: integration
  integration:
    provider: email
    setup:
      host: smtp.mailgun.org
      password: {mailgun_password}
      port: 587
      user: postmaster@email.julep.ai

- name: search_docs
  type: system
  system:
    resource: agent
    subresource: doc
    operation: search
    
main:
- prompt: |-
    f'''You are {{ agent.name }}. {{ agent.about }}

    A user with email address {{ _.from }} has sent the following inquiry:
    ------
      Subject: {{ _.subject }}

      {{ _.body }}
    ------

    Can you generate a query to search the documentation based on this email?
    Just respond with the query as is and nothing else.'''

  unwrap: true

- tool: search_docs
  arguments:
    agent_id: {agent.id}
    text: _
    
- prompt: >-
    f'''You are {{ agent.name }}. {{ agent.about }}

    A user with email address {{ steps[0].input.from }} has sent the following inquiry:
    ------
      Subject: {{ steps[0].input.subject }}

      {{ steps[0].input.body }}
    ------

    Here are some possibly relevant snippets from the julep documentation:
    {{ '\\n'.join([snippet.content for doc in _.docs for snippet in doc.snippets]) }}
    
    ========

    Based on the above info, craft an email body to respond with as a json object.
    The json object must have `subject` and `body` fields.'''
  response_format:
    type: json_object
    
  unwrap: true
  
- evaluate:
    subject: $ load_json(_.split('```json')[1].split('```')[0])['subject']
    body: $ load_json(_.split('```json')[1].split('```')[0])['body']
    
- tool: send_email
  arguments:
    body: $ _.body
    from: postmaster@email.julep.ai
    subject: $  _.subject
    to: $ steps[0].input['from']
""")

<span style="color:olive;">Notes:</span>
- The `unwrap: True` in the prompt step is used to unwrap the output of the prompt step (to unwrap the `choices[0].message.content` from the output of the model).


## Creating a task

In [28]:
task = julep.tasks.create(
    agent_id=agent.id,
    **task_def,
)

In [29]:
task.id

'067a5e5c-06d9-788f-8000-b35d78e72bc0'

## Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

In [30]:
execution = julep.executions.create(
    task_id=task.id,
    input={"from": "diwank@julep.ai", "to": "help@agents.new", "subject": "what's up", "body": "sup"},
)

In [31]:
julep.executions.get(execution.id)

Execution(id='067a5e5c-0aab-7227-8000-61965b0ea1c4', created_at=datetime.datetime(2025, 2, 7, 10, 51, 44, 667044, tzinfo=datetime.timezone.utc), input={'to': 'help@agents.new', 'body': 'sup', 'from': 'diwank@julep.ai', 'subject': "what's up"}, status='starting', task_id='067a5e5c-06d9-788f-8000-b35d78e72bc0', updated_at=datetime.datetime(2025, 2, 7, 10, 51, 44, 705438, tzinfo=datetime.timezone.utc), error=None, metadata={}, output={'to': 'help@agents.new', 'body': 'sup', 'from': 'diwank@julep.ai', 'subject': "what's up"})

## Checking execution details and output

There are multiple ways to get the execution details and the output:

1. **Get Execution Details**: This method retrieves the details of the execution, including the output of the last transition that took place.

2. **List Transitions**: This method lists all the task steps that have been executed up to this point in time, so the output of a successful execution will be the output of the last transition (first in the transition list as it is in reverse chronological order), which should have a type of `finish`.


<span style="color:olive;">Note: You need to wait for a few seconds for the execution to complete before you can get the final output, so feel free to run the following cells multiple times until you get the final output.</span>


In [35]:
execution_transitions = julep.executions.transitions.list(
    execution_id=execution.id).items

for transition in reversed(execution_transitions):
    print("Type: ", transition.type)
    print("output: ", transition.output)
    print("-" * 100)

Type:  init
output:  {'to': 'help@agents.new', 'body': 'sup', 'from': 'diwank@julep.ai', 'subject': "what's up"}
----------------------------------------------------------------------------------------------------
Type:  step
output:  I'm sorry, but I need more specific information from the email content to generate an appropriate query for the documentation search. Could you provide the subject and body of the email?
----------------------------------------------------------------------------------------------------
Type:  step
output:  {'docs': [], 'time': 0.0017147064208984375}
----------------------------------------------------------------------------------------------------
Type:  step
output:  To craft a response, I would need specific details from the user's inquiry and relevant snippets from the Julep documentation. Since I don't have access to the actual content of the inquiry or the documentation snippets, I'll provide a general template for how you might structure the JSON 